# Processing words as a set: The bag-of-words approach

## download the data

```sh
pdm run data_imdb
```

In [1]:
import os
import pathlib
import random
import shutil

In [2]:
base_dir = pathlib.Path("./data/acl_imdb/aclImdb")


def prepare_val_data():
    val_dir = base_dir / "val"
    if val_dir.exists():
        print("validation data already existed")
        return
    train_dir = base_dir / "train"
    for category in ("neg", "pos"):
        os.makedirs(val_dir / category)
        files = os.listdir(train_dir / category)
        random.Random(1337).shuffle(files)
        num_val_samples = int(0.2 * len(files))
        val_files = files[-num_val_samples:]
        for fname in val_files:
            shutil.move(train_dir / category / fname, val_dir / category / fname)


prepare_val_data()

In [3]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    f"{base_dir}/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    f"{base_dir}/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    f"{base_dir}/test", batch_size=batch_size
)

Found 16000 files belonging to 2 classes.
Metal device set to: Apple M1 Max


2023-02-10 22:18:21.764318: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-10 22:18:21.764457: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 4000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [4]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"This is film that was actually recommended to me by my dentist, and am I glad he did! The blend of British humor (should I say, Humour?) and the reality of a lost, middle-aged widow trying to maintain her lifestyle were a hoot. Add to that mix the reality of what it takes to actually grow pot (those plants under the bushes were NOT going to make it without the TLC they received), and it is a truly hilarious, yet touching film. I laugh every time I conjure the vision of all the bar patrons sitting in their lawn chairs with sunglasses on counting down the lights! Maybe it's just my Mendocino County blood, but the Brits definitely got this one right!! 10/10", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


2023-02-10 22:18:23.067032: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


## Processing words as a set: The bag-of-words approach

### Single words (unigrams) with binary encoding

In [5]:
CPU_CORES = 8

from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)

2023-02-10 22:18:23.132596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


inspect the encoded dataset

In [6]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


model building utils

In [7]:
from tensorflow import keras
from tensorflow.keras import layers


def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [8]:
model = get_model()
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint("binary_1gram.keras", save_best_only=True)]
model.fit(
    binary_1gram_train_ds.cache(),
    validation_data=binary_1gram_val_ds.cache(),
    epochs=10,
    callbacks=callbacks,
)
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
  1/500 [..............................] - ETA: 2:32 - loss: 0.7003 - accuracy: 0.4688

2023-02-10 22:18:24.676441: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


496/500 [============================>.] - ETA: 0s - loss: 0.4129 - accuracy: 0.8267

2023-02-10 22:18:28.599495: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 5s 9ms/step - loss: 0.4126 - accuracy: 0.8271 - val_loss: 0.3173 - val_accuracy: 0.8718
Epoch 2/10
500/500 [==============================] - 4s 8ms/step - loss: 0.1987 - accuracy: 0.9301 - val_loss: 0.3008 - val_accuracy: 0.8823
Epoch 3/10
500/500 [==============================] - 4s 9ms/step - loss: 0.1235 - accuracy: 0.9602 - val_loss: 0.3217 - val_accuracy: 0.8845
Epoch 4/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0837 - accuracy: 0.9745 - val_loss: 0.3617 - val_accuracy: 0.8863
Epoch 5/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0600 - accuracy: 0.9829 - val_loss: 0.4130 - val_accuracy: 0.8835
Epoch 6/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0457 - accuracy: 0.9873 - val_loss: 0.4731 - val_accuracy: 0.8798
Epoch 7/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0366 - accuracy: 0.9902 - val_loss: 0.5333 - val_accuracy: 0.8775
Epoch 8/10
500/50

2023-02-10 22:19:06.046689: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 5s 6ms/step - loss: 0.2965 - accuracy: 0.8816
Test acc: 0.882


### Bigrams with binary encoding

In [9]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

text_vectorization.adapt(text_only_train_ds)

binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)

2023-02-10 22:19:10.937147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [10]:
model = get_model()
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint("binary_2gram.keras", save_best_only=True)]
model.fit(
    binary_2gram_train_ds.cache(),
    validation_data=binary_2gram_val_ds.cache(),
    epochs=10,
    callbacks=callbacks,
)
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
  6/500 [..............................] - ETA: 5s - loss: 0.6850 - accuracy: 0.5469  

2023-02-10 22:19:16.965442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


496/500 [============================>.] - ETA: 0s - loss: 0.3893 - accuracy: 0.8368

2023-02-10 22:19:20.797163: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 5s 9ms/step - loss: 0.3895 - accuracy: 0.8371 - val_loss: 0.2870 - val_accuracy: 0.8818
Epoch 2/10
500/500 [==============================] - 4s 7ms/step - loss: 0.1540 - accuracy: 0.9499 - val_loss: 0.2820 - val_accuracy: 0.8940
Epoch 3/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0747 - accuracy: 0.9793 - val_loss: 0.3211 - val_accuracy: 0.8973
Epoch 4/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0389 - accuracy: 0.9892 - val_loss: 0.3774 - val_accuracy: 0.8955
Epoch 5/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0222 - accuracy: 0.9943 - val_loss: 0.4493 - val_accuracy: 0.8960
Epoch 6/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0136 - accuracy: 0.9966 - val_loss: 0.5130 - val_accuracy: 0.8945
Epoch 7/10
500/500 [==============================] - 4s 9ms/step - loss: 0.0090 - accuracy: 0.9979 - val_loss: 0.5820 - val_accuracy: 0.8948
Epoch 8/10
500/50

2023-02-10 22:19:57.950047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 5s 6ms/step - loss: 0.2726 - accuracy: 0.8977
Test acc: 0.898


### Bigrams with TF-IDF encoding

In [11]:
text_vectorization = TextVectorization(ngrams=2, max_tokens=20000, output_mode="tf_idf")

text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)

2023-02-10 22:20:03.035292: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [12]:
model = get_model()
model.summary()
callbacks = [keras.callbacks.ModelCheckpoint("tfidf_2gram.keras", save_best_only=True)]
model.fit(
    tfidf_2gram_train_ds.cache(),
    validation_data=tfidf_2gram_val_ds.cache(),
    epochs=10,
    callbacks=callbacks,
)
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
  7/500 [..............................] - ETA: 4s - loss: 2.5526 - accuracy: 0.5357  

2023-02-10 22:20:21.261065: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


493/500 [============================>.] - ETA: 0s - loss: 0.5143 - accuracy: 0.7603

2023-02-10 22:20:25.088540: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 5s 9ms/step - loss: 0.5147 - accuracy: 0.7611 - val_loss: 0.3746 - val_accuracy: 0.8413
Epoch 2/10
500/500 [==============================] - 4s 9ms/step - loss: 0.2603 - accuracy: 0.8967 - val_loss: 0.3595 - val_accuracy: 0.8315
Epoch 3/10
500/500 [==============================] - 4s 8ms/step - loss: 0.1545 - accuracy: 0.9362 - val_loss: 0.3723 - val_accuracy: 0.8623
Epoch 4/10
500/500 [==============================] - 4s 8ms/step - loss: 0.1055 - accuracy: 0.9526 - val_loss: 0.4310 - val_accuracy: 0.8605
Epoch 5/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0745 - accuracy: 0.9629 - val_loss: 0.5131 - val_accuracy: 0.8653
Epoch 6/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0596 - accuracy: 0.9704 - val_loss: 0.6373 - val_accuracy: 0.8840
Epoch 7/10
500/500 [==============================] - 4s 8ms/step - loss: 0.0445 - accuracy: 0.9783 - val_loss: 0.6968 - val_accuracy: 0.8808
Epoch 8/10
500/50

2023-02-10 22:21:03.866288: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 5s 6ms/step - loss: 0.3471 - accuracy: 0.8311
Test acc: 0.831


Exporting a model that processes raw strings

In [13]:
inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

In [14]:
import tensorflow as tf

raw_text_data = tf.convert_to_tensor(
    [
        ["That was an excellent movie, I loved it."],
    ]
)
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

94.28 percent positive
